## Introduction

SMT (Satisfiability Modulo Theory) solvers allow us to solve formulas involving not only propositional
variables, but also variables from a given universe (sort), e.g., reals or integers. Quantifiers are also
supported. Z3 is both a SAT solver and a SMT solver.
In this lab, we will be working with the SMT-LIB input format (version 2). This format is 
inspired by the programming language LISP, and supported 
by many SMT solvers, thus allowing us to easily test multiple solvers on our formulas.

## How to run

The commands below can be entered into z3 interactively. Run `z3 -smt2 -in` to enter commands into z3 (even better, use `rlwrap z3 -smt2 -in` to allow line editing). The magic line `%%script z3 -smt2 -in` lets us to send the commands to z3 directly from Jupyter.

## SMT solver as a calculator

SMT-LIB does not use 
typical expression format (with unary and binary operators); instead, expressions and commands are given as
`(operator <parameter> ... <parameter>)`, similar to the programming language LISP. Thus, 
`x>y` is written as `(> x y)`. This LISP-like format is very easy to parse for a computer.

Enter the following commands by hand:


In [1]:
%%script z3 -smt2 -in

(simplify (+ 8 12))
(simplify (> 8 5))

20
true


`simplify` is a command which tries to simplify the given expression.

## Constants

In [2]:
%%script z3 -smt2 -in

(declare-const x Real)
(define-const y Real 6.0)

;# (comments in SMTLIB format start with ;, we also add # so that they look correctly in Jupyter)

;# Here, both `x` and `y` are real constants. 
;# Available sorts in Z3 include Real, Bool, and Int.
;# `x` is declared, i.e., no interpretation is provided.
;# `y` is defined, i.e., an interpretation is provided.

(simplify (+ x x y y))

(simplify (* (+ x 1) (+ x 1)))


(+ 12.0 (* 2.0 x))
(* (+ 1.0 x) (+ 1.0 x))


**Exercise:** Simplify does not simplify polynomials as sums of monomials by default. Use `(help simplify)` to find an option which changes this. Options are given e.g. as in `(simplify (+ x x) :timeout 500)`.




## Checking satisfiability

Suppose we want to check whether the formula `x>y` is satisfiable in reals.


In [9]:
%%script z3 -smt2 -in

(reset)

;# Reset Z3 to forget the previous declarations of `x` and `y`.
;# (This is not needed when you are just running from Jupyter -- z3 is restarted each time)

(declare-const x Real)
(declare-const y Real)

;# We need to declare our constants

(assert (> x y))

;# We add our formula as an assertion

(check-sat)

;# We check whether our assertion is satisfiable. Z3 will try to tell whether there is
;# an interpretation of declared, but undefined constants `x` and `y` such that our formula
;# is satisfied. In this case, such an interpretation exists, so Z3 says `sat`.

(get-value (x y))

;# This tells us the values of `x` and `y` in the model found. We can also perform
;# computations on them, e.g., `(get-value ((+ x y)))`.



sat
((x 0.0)
 (y (- 1.0)))


**Exercise:** Use Z3 to find integers $x, y$ such that $x,y>1$ and $xy = 1001$.

**Exercise:** Use Z3 to find integer $x$ such that $x \equiv 20\ (\rm{mod\ }101)$ and $x \equiv 80\ (\rm{mod\ }103)$. Z3 has a mod operator, but avoid using it by introducing extra variables.

**Exercise**: Diophantine equations (i.e., ones where we are looking for integer solutions) are undecidable in general, but Z3 is able to solve these particular ones. Guess if Z3 is able to check whether the following are satisfiable in integers: $x^2+y^2=z^2$ ($x,y>1$); $x^3+y^3=z^3$ ($x,y>1$); $x \equiv 20\ ({\rm mod\ }1000000002)$ and $x \equiv 81\ ({\rm mod}\ 1000000005)$; $x^2+y^2=4z+3$. Verify your suspicions.



In [8]:
%%script z3 -smt2 -in
;# Exercise 1 - to może być faktoryzacja
(reset)

(declare-const x Real)
(declare-const y Real)

(assert (> x 1))
(assert (> y 1))
(assert (= 1001 (* x y)))

(check-sat)

(get-value (x y))

sat
((x 2.0)
 (y (/ 1001.0 2.0)))


In [19]:
%%script z3 -smt2 -in
;# Exercise 2

(reset)

(declare-const x Int)
(declare-const m1 Int)
(declare-const m2 Int)

(assert (= x (+ 20 (* 101 m1))))
(assert (< m1 101))
(assert (= x (+ 80 (* 103 m2))))
(assert (< m2 103))
(assert (<= 0 x))

(check-sat)

(get-value (x))

sat
((x 7393))


In [25]:
%%script z3 -smt2 -in
;# Exercise 3

(reset)

(declare-const x Int)
(declare-const y Int)
(declare-const z Int)
(declare-const m1 Int)
(declare-const m2 Int)

; Formula case 1
; (assert (> x 1))
; (assert (> y 1))
; (assert (= (+ (* x x) (* y y)) (* z z)))
; Sat

; Formula case 2
; (assert (> x 1))
; (assert (> y 1))
; (assert (= (+ (* x x x) (* y y y)) (* z z z)))
; Unknown

; Formula case 3
;(assert (= x (+ 20 (* 1000000002 m1))))
;(assert (< m1 1000000002))
;(assert (= x (+ 81 (* 1000000005 m2))))
;(assert (< m2 1000000005))
; Unsat

; Formula case 4
; (assert (> x 1))
; (assert (> y 1))
; (assert (= (+ (* x x) (* y y)) (+ (* 4 z) 3)))
; Unknown

(check-sat)

(get-value (x y z))

unknown
(error "line 38 column 18: model is not available")


## Incremental solving

In the last section, we had to reset Z3 to remove the definitions and assertions which we no longer wanted. 
This may be inconvenient when we want to keep a part of the assertions. To solve this, we can use Z3's internal stack.
Type:

In [4]:
%%script z3 -smt2 -in

(reset)
(declare-const x Real)
(declare-const y Real)
(declare-const z Real)
(assert (> x y))
(assert (> y z))
(check-sat)

;# Here, z3 says the formula is satisfiable. Let us check if it is still satisfiable
;# when we additionally assume that `z>x`.

(push)
(assert (> z x))
(check-sat)

;# It is not, so we want to remove the constraint `z>x` (no point to add new constraint if it is already unsatisfiable).
;# Luckily, we have pushed the current solver state with `(push)` before adding this
;# contraint, so we can use `(pop)` to get back to that state, and try a new constraint, e.g.:

(pop)
(push)
(assert (= z (+ x 3)))
(check-sat)

;# This was done interactively. An external program could redirect input and output to Z3.

sat
unsat
unsat


**Exercise:** A group of children has visited the local candy store. A cake costs 3.7 PLN, a donut costs 1.2 PLN, an ice cream with one scoop costs 3.0 PLN, and 2.5 PLN more for each extra scoop. Krzysiek has paid exactly 9.7 PLN. Jaś and Małgosia both have paid exactly 7.9 PLN; Jaś hates cakes (never buys them), but Małgosia loves them (and always buys them). Every child has bought at most one icecream. Use Z3 to find out what every child could have bought. Use (push) and (pop) to share the assertions which are common between all children. The function `ite` could be useful (if-then-else: (ite x y z) is similar to x?y:z in C).

In [33]:
%%script z3 -smt2 -in

(reset)

(declare-const cake Int)
(declare-const donut Int)
(declare-const icecream Int)
(declare-const cost Real)

(assert (>= cake 0))
(assert (>= donut 0))
(assert (>= icecream 0))

(assert (= cost (+ (* 3.7 cake) (* 1.2 donut) (ite (= icecream 0) 0 (+ 3.0 (* 2.5 (- icecream 1) ) )))))

; Wynik dla Krzyśka
(push)
(assert (= cost 9.7))
(check-sat)
(get-value (cake donut icecream))
(pop)

; Wynik dla Jasia
(push)
(assert (= cost 7.9))
(assert (= cake 0))
(check-sat)
(get-value (cake donut icecream))
(pop)

; Wynik dla Małgosi
(push)
(assert (= cost 7.9))
(assert (> cake 0))
(check-sat)
(get-value (cake donut icecream))
(pop)

sat
((cake 1)
 (donut 5)
 (icecream 0))
sat
((cake 0)
 (donut 2)
 (icecream 2))
sat
((cake 1)
 (donut 1)
 (icecream 1))


## Quantifiers

In the example below, we check whether the order on reals is dense. The new elements include implication `=>` and quantifiers `forall` and `exists`.


In [5]:
%%script z3 -smt2 -in

(assert 
  (forall 
    ((x Real) (y Real)) 
    (=> 
      (< x y) 
      (exists 
        ((z Real)) 
        (and (< x z) (< z y))
      )
    )
  )
)
(check-sat)


sat


**Exercise:** Check if the order on integers is dense. Find a counterexample (i.e., x and y such that z does not exist).


In [46]:
%%script z3 -smt2 -in

; Checking if integers are dense
(reset)

(assert 
  (forall 
    ((x Int) (y Int)) 
    (=> 
      (< x y) 
      (exists 
        ((z Int)) 
        (and (< x z) (< z y))
      )
    )
  )
)
(check-sat)

; Getting counterexample
(reset)

(declare-const x Int)
(declare-const y Int)

(assert (< x y))

(assert (not (exists ((z Int)) (and (< x z) (< z y)))))

(check-sat)
(get-value (x y))

unsat
sat
((x 0)
 (y 1))


## Functions

We can also declare/define function symbols.


In [43]:
%%script z3 -smt2 -in

(declare-fun f (Real Real) Real)
;# We declare a function symbol `f`, with two Real arguments and Real value.

(define-fun assoc () Bool (forall ((x Real) (y Real) (z Real)) (= (f (f x y) z) (f x (f y z)))))
;# We define a function symbol `assoc`, with no arguments and Bool value, i.e., a boolean constant.
;# We define assoc to say that `f` is associative. This way, we can refer to the associativity condition via `assoc`.

;# Let us check whether an associative `f` exists:

(push)
(assert assoc)
(check-sat)
(get-model)
(pop)

;# Let us check whether an non-associative `f` exists:

(push)
(assert (not assoc))
(check-sat)
(get-model)
(pop)

;# We use `(get-model)` to retrieve the examples found by Z3.

sat
(model 
  (define-fun f ((x!0 Real) (x!1 Real)) Real
    0.0)
)
sat
(model 
  (define-fun x!6 () Real
    0.0)
  (define-fun z!4 () Real
    3.0)
  (define-fun y!5 () Real
    1.0)
  (define-fun f ((x!0 Real) (x!1 Real)) Real
    (ite (and (= x!0 0.0) (= x!1 1.0)) 2.0
    (ite (and (= x!0 2.0) (= x!1 3.0)) 4.0
    (ite (and (= x!0 1.0) (= x!1 3.0)) 5.0
    (ite (and (= x!0 0.0) (= x!1 5.0)) 6.0
      2.0)))))
)


**Exercise:** Check whether the following functions are associative: +, -, max, by adding an assertion that
`f` is the given function.

In [56]:
%%script z3 -smt2 -in
(reset)

(declare-fun f (Real Real) Real)

(define-fun assoc () Bool (forall ((x Real) (y Real) (z Real)) (= (f (f x y) z) (f x (f y z)))))
(assert assoc)

(push)
(define-fun f2 ((x Real) (y Real)) Real (+ x y) )
(assert (= f f2))
(check-sat)
(get-model)
(pop)

(error "line 10 column 11: invalid function application, missing arguments f")
sat
(model 
  (define-fun f ((x!0 Real) (x!1 Real)) Real
    0.0)
)


**Exercise:** Find a function which is commutative, but not associative. For what arguments `x`, `y`, `z` is it not associative?

**Exercise:** Try to find a function which is associative, but not commutative (Z3 4.6.0 has problems with this).

In [57]:
%%script z3 -smt2 -in

(declare-fun f (Real Real) Real)

(define-fun assoc () Bool (forall ((x Real) (y Real) (z Real)) (= (f (f x y) z) (f x (f y z)))))
(define-fun comm () Bool (forall ((x Real) (y Real)) (= (f x y) (f y x))))

(assert assoc)
(assert (not comm))
(check-sat)
(get-model)

Process is interrupted.


## Warning: division by zero

All functions in Z3 are total. This includes division: since all functions are total, 1/0 exists, but it is not specified.


In [17]:
%%script z3 -smt2 -in

(assert (= (/ 1 0) 10))
(check-sat)



sat


## Declaring new sorts

In the example below, we create a new sort `Z` (0 means that this sort is not parametrized; there can be parametrized sorts, e.g., a sort of arrays indexed with sort `S1` and with values in sort `S2`). `(check-sat)` tells us the formula is satisfiable, and `(get-model)` gives us a model with 2 elements.

In [28]:
%%script z3 -smt2 -in

(declare-sort Z 0)
(declare-const x Z)
(assert 
  (exists 
    ((z Z)) 
    (and 
      (not (= x z))
      (forall ((y Z)) (or (= y x) (= y z)))
    )
  )
)
(check-sat)
(get-value (x))
(get-model)


sat
((x Z!val!0))
(model 
  ;; universe for Z:
  ;;   Z!val!0 Z!val!1 
  ;; -----------
  ;; definitions for universe elements:
  (declare-fun Z!val!0 () Z)
  (declare-fun Z!val!1 () Z)
  ;; cardinality constraint:
  (forall ((x Z)) (or (= x Z!val!0) (= x Z!val!1)))
  ;; -----------
  (define-fun z!0 () Z
    Z!val!1)
  (define-fun x () Z
    Z!val!0)
)
